# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [53]:
# import dependencies 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint
import json
from ipywidgets.embed import embed_minimal_html # for export of widgets

# Import Google API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# read csv of cities data from weatherpy
file_location = "../output_data/city_data.csv"
city_data = pd.read_csv(file_location)
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Oranjemund,-28.5500,16.4333,18.82,86,13,2.52,NaN,1610593174
1,Yellowknife,62.4560,-114.3525,-11.00,92,75,4.12,CA,1610593022
2,Wanning,18.8003,110.3967,19.36,64,55,3.91,CN,1610593175
3,Busselton,-33.6500,115.3333,21.11,46,100,1.79,AU,1610593176
4,Rikitea,-23.1203,-134.9692,25.51,77,71,5.19,PF,1610593178
...,...,...,...,...,...,...,...,...,...
543,Tahe,52.3333,124.7333,-22.85,82,9,1.16,CN,1610593680
544,Gōtsuchō,35.0000,132.2167,12.00,62,20,3.09,JP,1610593681
545,Shahrud,36.4182,54.9763,2.17,71,0,1.96,IR,1610593442
546,Awjilah,29.1081,21.2869,12.23,60,65,7.83,LY,1610593683


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [40]:
gmaps.configure(api_key = g_key)

In [65]:
# set gmap values
locations = city_data_df[["Latitude", "Longitude"]].astype(float)
humidity = city_data_df["Humidity"].astype(float)

# use values and display a heat map
fig = gmaps.figure(center = (0.0, 0.0), zoom_level=1)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity)
fig.add_layer(heat_layer)

# export results for offline viewing
# https://jupyter-gmaps.readthedocs.io/en/latest/export.html
embed_minimal_html("../output_data/heatmap.html", views=[fig]) 

fig

A Jupyter Widget

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 32 but higher than 25 degrees celsius.
# Wind speed less than 10 mph.
# Zero cloudiness.

ideal_weather = city_data_df.loc[((city_data_df["Max Temp"] > 25) & (city_data_df["Max Temp"] < 32))
                                 & (city_data_df["Wind Speed"] <= 10)
                                 & (city_data_df["Cloudiness"] == 0) 
                                   ]
ideal_weather

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
206,Kilindoni,-7.9139,39.6668,26.47,79,0,5.38,TZ,1610593395
354,Reporoa,-38.4333,176.3500,30.56,36,0,1.79,NZ,1610593526
361,Acajutla,13.5928,-89.8275,26.22,70,0,2.41,SV,1610593297
446,Port Blair,11.6667,92.7500,27.14,66,0,5.10,IN,1610593605
448,Bubaque,11.2833,-15.8333,25.31,69,0,3.47,GW,1610593606
456,Byron Bay,-28.6500,153.6167,28.89,50,0,4.63,AU,1610593613
462,Bonthe,7.5264,-12.5050,25.45,87,0,1.96,SL,1610593618
483,Port Augusta,-32.5000,137.7667,27.22,56,0,7.20,AU,1610593634


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
# add blank column
hotel_df = ideal_weather
hotel_df["Hotel Name"] = ""
hotel_df

<ipython-input-24-29a6706857e5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
206,Kilindoni,-7.9139,39.6668,26.47,79,0,5.38,TZ,1610593395,
354,Reporoa,-38.4333,176.3500,30.56,36,0,1.79,NZ,1610593526,
361,Acajutla,13.5928,-89.8275,26.22,70,0,2.41,SV,1610593297,
446,Port Blair,11.6667,92.7500,27.14,66,0,5.10,IN,1610593605,
448,Bubaque,11.2833,-15.8333,25.31,69,0,3.47,GW,1610593606,
456,Byron Bay,-28.6500,153.6167,28.89,50,0,4.63,AU,1610593613,
462,Bonthe,7.5264,-12.5050,25.45,87,0,1.96,SL,1610593618,
483,Port Augusta,-32.5000,137.7667,27.22,56,0,7.20,AU,1610593634,


In [25]:
hotel_df = hotel_df.reset_index(drop=True)
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Kilindoni,-7.9139,39.6668,26.47,79,0,5.38,TZ,1610593395,
1,Reporoa,-38.4333,176.3500,30.56,36,0,1.79,NZ,1610593526,
2,Acajutla,13.5928,-89.8275,26.22,70,0,2.41,SV,1610593297,
3,Port Blair,11.6667,92.7500,27.14,66,0,5.10,IN,1610593605,
4,Bubaque,11.2833,-15.8333,25.31,69,0,3.47,GW,1610593606,
5,Byron Bay,-28.6500,153.6167,28.89,50,0,4.63,AU,1610593613,
6,Bonthe,7.5264,-12.5050,25.45,87,0,1.96,SL,1610593618,
7,Port Augusta,-32.5000,137.7667,27.22,56,0,7.20,AU,1610593634,


In [59]:
# https://developers.google.com/maps/documentation/javascript/examples/places-autocomplete-hotelsearch
# use lodging instead of the term hotel according to documentation above

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# loop over index/rows in hotel_df to get lat and longitudes for request 
for index, row in hotel_df.iterrows():
    longitude = hotel_df["Longitude"][index]
    latitude = hotel_df["Latitude"][index]
    location = f"{latitude},{longitude}"
    
    # parameters for search
    params = {"location": location, 
              "radius": 5000, 
              "types": "lodging",
              "key": g_key}
    
    # Search and retrieve json from query request
    response = requests.get(base_url, params)
    response_json = response.json()
    # print(json.dumps(response_json, indent =4, sort_keys =True))
    
    try:
        # put the result into column
        hotel_df.loc[index, "Hotel Name"] = response_json["results"][2]["name"]
    except:
        # if no result found print nothing found
        hotel_df.loc[index, "Hotel Name"] = "Nothing Found!" 
   

In [60]:
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Kilindoni,-7.9139,39.6668,26.47,79,0,5.38,TZ,1610593395,Kilindoni Magemani B&B
1,Reporoa,-38.4333,176.3500,30.56,36,0,1.79,NZ,1610593526,Nothing Found!
2,Acajutla,13.5928,-89.8275,26.22,70,0,2.41,SV,1610593297,Residencial la linda vista
3,Port Blair,11.6667,92.7500,27.14,66,0,5.10,IN,1610593605,J Hotel
4,Bubaque,11.2833,-15.8333,25.31,69,0,3.47,GW,1610593606,Apart Hotel New Neguen
5,Byron Bay,-28.6500,153.6167,28.89,50,0,4.63,AU,1610593613,Cape Byron YHA
6,Bonthe,7.5264,-12.5050,25.45,87,0,1.96,SL,1610593618,Nothing Found!
7,Port Augusta,-32.5000,137.7667,27.22,56,0,7.20,AU,1610593634,Discovery Parks - Port Augusta


In [61]:
# create data frame with only cities that have hotels found
hotels_found = hotel_df.loc[hotel_df["Hotel Name"] != "Nothing Found!"]
hotels_found

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Kilindoni,-7.9139,39.6668,26.47,79,0,5.38,TZ,1610593395,Kilindoni Magemani B&B
2,Acajutla,13.5928,-89.8275,26.22,70,0,2.41,SV,1610593297,Residencial la linda vista
3,Port Blair,11.6667,92.7500,27.14,66,0,5.10,IN,1610593605,J Hotel
4,Bubaque,11.2833,-15.8333,25.31,69,0,3.47,GW,1610593606,Apart Hotel New Neguen
5,Byron Bay,-28.6500,153.6167,28.89,50,0,4.63,AU,1610593613,Cape Byron YHA
7,Port Augusta,-32.5000,137.7667,27.22,56,0,7.20,AU,1610593634,Discovery Parks - Port Augusta


In [62]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_found.iterrows()]
ideal_locations = hotels_found[["Latitude", "Longitude"]]

In [64]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

# heatmap
fig.add_layer(gmaps.heatmap_layer(locations, weights = humidity))
# markers with hotel name,city and country
fig.add_layer(gmaps.marker_layer(ideal_locations, info_box_content = hotel_info))

# export results for offline viewing
# https://jupyter-gmaps.readthedocs.io/en/latest/export.html
embed_minimal_html("../output_data/hotel_markers.html", views=[fig])

# Display figure
fig

A Jupyter Widget